### Курсовой проект.
Разработка модели

In [292]:
# С начала курса хотелось поработать с анализом текстовой информации
# В поисках темы наткнулся на датасет на Kaggle, содержащий рецензии на кинофильмы. 
# https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews?select=IMDB+Dataset.csv
# С размеченными оценками окрасок рецензий (негативная/позитивная)
# Тема заинтересовала, но датасет содержал разумеется только англоязычные тексты, 
# а хотелось чего-нибудь более релевантного нашей действительности.
# В итоге проект начался с написания "сборщика" рецензий с "Кинопоиска" - благо окраска рецензии там уже проставлена
# В процессе сбора информации было получено около 1700 рецензий 
# (возможно с повторениями - если фильм входил сразу в несколько категорий)
# Эмоциональная окраска там не бинарна - а имеет три состояния (плохо/нейтрально/хорошо)
# Соответственно это не будет задача бинарной классификации. 
# А будет предсказание непрерывного значения от 0 до 1 (0 - плохо, 0.5 - нейтрально, 1 - хорошо)

In [3]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import xgboost as xg 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.naive_bayes import MultinomialNB

In [4]:
data = pd.read_csv('kinopoisk.csv')
data.head(3)

,Unnamed: 0,grade,text
0,0,1.0,"Скажу сразу, 'Брат 2' мне очень нравится, смот..."
1,1,0.0,Наверное не стоит начинать рецензию с экскурса...
2,2,1.0,Европейский язык искусства всегда был самостоя...


In [3]:
data.shape

(1699, 3)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1699 entries, 0 to 1698
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1699 non-null   int64  
 1   grade       1699 non-null   float64
 2   text        1699 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 39.9+ KB


In [5]:
# Проверим распределение целевой переменной
data['grade'].value_counts(normalize=True)

1.0    0.729253
0.5    0.145380
0.0    0.125368
Name: grade, dtype: float64

In [15]:
# Довольно сильный перекос в сторону "хвалебных" отзывов, надо-же, а когда читаешь Кинопоиск, 
# кажется, что российские критики (и российские зрители) - на 99% убежденные хейтеры.
# Видимо я в основном на наши фильмы читал рецензии ))

In [7]:
data['grade'].value_counts(normalize=False)

1.0    1239
0.5     247
0.0     213
Name: grade, dtype: int64

In [17]:
# Ну - попробуем
X_train, X_test, y_train, y_test = train_test_split(data, data['grade'], random_state=0, test_size=0.25)

In [69]:
text_transformer = Pipeline([
    ('tfidf', TfidfVectorizer()),
])

In [101]:
feat_prep = ColumnTransformer(
        transformers=[
            # ('dummy', 'passthrough', ['Unnamed: 0']),
            ('text', text_transformer, ['text']),
            ]
            ,remainder='drop'
)

In [102]:
linreg = make_pipeline(feat_prep)

In [105]:
linreg.fit_transform(X_train)
# vectorizer = TfidfVectorizer()

array([[1.]])

In [93]:
X_train.shape, y_train.shape

((1274, 3), (1274,))

In [52]:
X.shape

(3, 3)

In [50]:
vectorizer

TfidfVectorizer()

In [79]:
X_train['Unnamed: 0']

1310    1310
215      215
1170    1170
1508    1508
536      536
        ... 
835      835
1216    1216
1653    1653
559      559
684      684
Name: Unnamed: 0, Length: 1274, dtype: int64

In [148]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]


pipeline = Pipeline([('title_selector', FeatureSelector(column='text')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('reg', LinearRegression())
                    ])

In [143]:
preds = pipeline.predict(X_test)
r2_score(y_test, preds)

0.23331542854562737

In [204]:
pipeline = Pipeline([('title_selector', FeatureSelector(column='text')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('reg', Ridge(alpha=0.07))
                    ])

In [205]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
r2_score(y_test, preds)

0.2864044817022402

In [172]:
pipeline = Pipeline([('title_selector', FeatureSelector(column='text')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('reg', xg.XGBRegressor(n_estimators = 10, seed = 123))
                    ])

In [173]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
r2_score(y_test, preds)

0.10352648306667522

In [155]:
pipeline = Pipeline([('title_selector', FeatureSelector(column='text')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('reg', RandomForestRegressor())
                    ])

In [156]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
r2_score(y_test, preds)

0.1440027255781695

In [ ]:
# Эх.. чуда ML не случилось (( а какие были перспективы...
# Точность моделей на уровне плинтуса
# Признак - по сути один, какой тут возможен feature engineering - мне неведомо
# Погоняем еще немного параметры на лучшем варианте модели (polynomial)

In [214]:
pipeline = Pipeline([('title_selector', FeatureSelector(column='text')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('reg', Ridge())
                    ])
parms = {'reg__solver': ('auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'), 'reg__alpha': (0, 0.05, 0.1, 0.2, 0.5, 1)}
gs = GridSearchCV(pipeline, parms, scoring='r2', cv=KFold(n_splits=5, random_state=100, shuffle=True), n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
             estimator=Pipeline(steps=[('title_selector',
                                        FeatureSelector(column='text')),
                                       ('title_tfidf', TfidfVectorizer()),
                                       ('reg', Ridge())]),
             n_jobs=-1,
             param_grid={'reg__alpha': (0, 0.05, 0.1, 0.2, 0.5, 1),
                         'reg__solver': ('auto', 'svd', 'cholesky', 'lsqr',
                                         'sparse_cg', 'sag', 'saga')},
             scoring='r2')

In [215]:
gs.best_score_

0.2342401235288075

In [216]:
gs.best_params_

{'reg__alpha': 0.05, 'reg__solver': 'auto'}

In [253]:
# Вобщем - ничего нового
pipeline = Pipeline([('title_selector', FeatureSelector(column='text')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('reg', Ridge(alpha=0.1, tol=1e-16))
                    ])

In [254]:
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)
r2_score(y_test, preds)

0.2860449029981992

In [255]:
preds = pipeline.predict(X_train)
r2_score(y_train, preds)

0.9914154590629911

In [256]:
# Переобучение и низкие результаты на тесте.. 
# Возможно все-таки имеет смысл передумать насчет регрессии в сторону классификации

In [5]:
label_encoder = LabelEncoder()
data_labelled = data.copy()
data_labelled['grade'] = label_encoder.fit_transform(data['grade'])
data_labelled.head()

,Unnamed: 0,grade,text
0,0,2,"Скажу сразу, 'Брат 2' мне очень нравится, смот..."
1,1,0,Наверное не стоит начинать рецензию с экскурса...
2,2,2,Европейский язык искусства всегда был самостоя...
3,3,2,Неожиданно американская школьная комедия оказа...
4,4,2,«Титаник» разобран на штампы и цитаты с такой ...


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data_labelled, data_labelled['grade'], random_state=0, test_size=0.25)

In [285]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [288]:
pipeline.fit(X_train['text'].values, y_train.values)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [289]:
preds = pipeline.predict(X_test['text'].values)
np.mean(preds == y_test.values)

0.72

In [290]:
# Ну конечно так нужно было сразу и делать.. 

In [300]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-4, random_state=100,
                        max_iter=50, tol=None)),
])

In [301]:
pipeline.fit(X_train['text'].values, y_train.values)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(max_iter=50, random_state=100, tol=None))])

In [302]:
preds = pipeline.predict(X_test['text'].values)
np.mean(preds == y_test.values)

0.7435294117647059

In [319]:
# Прогоним подбор параметров
parms = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3, 1e-4, 1e-5),
    'clf__max_iter': (10, 50, 100),
}
gs = GridSearchCV(pipeline, parms, cv=KFold(n_splits=5, random_state=100, shuffle=True), n_jobs=-1)
gs.fit(X_train['text'].values, y_train.values)

GridSearchCV(cv=KFold(n_splits=5, random_state=100, shuffle=True),
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        SGDClassifier(max_iter=50,
                                                      random_state=100,
                                                      tol=None))]),
             n_jobs=-1,
             param_grid={'clf__alpha': (0.01, 0.001, 0.0001, 1e-05),
                         'clf__max_iter': (10, 50, 100),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2)]})

In [320]:
gs.best_score_

0.7519345375945654

In [321]:
gs.best_params_

{'clf__alpha': 0.0001,
 'clf__max_iter': 100,
 'tfidf__use_idf': False,
 'vect__ngram_range': (1, 2)}

In [362]:
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-4, random_state=100,
                        max_iter=100, tol=None)),
])

In [363]:
pipeline.fit(X_train['text'].values, y_train.values)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf',
                 SGDClassifier(max_iter=100, random_state=100, tol=None))])

In [364]:
preds = pipeline.predict(X_test['text'].values)
np.mean(preds == y_test.values)

0.7623529411764706

In [365]:
preds = pipeline.predict(X_train['text'].values)
np.mean(preds == y_train.values)

0.9992150706436421

In [ ]:
# Вобщем при низком alfa довольно отчетливо переобучается, хотя и тестовый скор растет. 
# Но видимо не стоит так сильно душить регуляризацию, может зато по-стабильнее будет реальный скор

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(use_idf=False)),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                        alpha=5e-4, random_state=100,
                        max_iter=150, tol=None)),
])

In [8]:
pipeline.fit(X_train['text'].values, y_train.values)

Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf',
                 SGDClassifier(alpha=0.0005, max_iter=150, random_state=100,
                               tol=None))])

In [412]:
preds = pipeline.predict(X_test['text'].values)
np.mean(preds == y_test.values)

0.7482352941176471

In [413]:
preds = pipeline.predict(X_train['text'].values)
np.mean(preds == y_train.values)

0.9897959183673469

In [414]:
preds = pipeline.predict(X_test['text'].values)
print(metrics.classification_report(y_test.values, preds, target_names=('negative', 'neutral', 'positive')))

              precision    recall  f1-score   support

    negative       0.68      0.23      0.34        57
     neutral       0.50      0.02      0.03        62
    positive       0.75      0.99      0.86       306

    accuracy                           0.75       425
   macro avg       0.65      0.41      0.41       425
weighted avg       0.71      0.75      0.67       425



In [415]:
metrics.confusion_matrix(y_test.values, preds)

array([[ 13,   0,  44],
       [  5,   1,  56],
       [  1,   1, 304]])

In [423]:
# roc_auc_score(pipeline.predict_proba(X_test['text'].values)[:, 1], y_test.values)

In [420]:
y_test.value_counts(normalize=False)

2    306
1     62
0     57
Name: grade, dtype: int64

In [416]:
# Вобщем, хуже всего классифицируются нейтральные отзывы, слабо распоздаются - негативные, лучше всего - позитивные
# Наверное не удивительно - у нас сильный перекос по количеству в сторону позитивных - в учебном датасете

### Ну - пока на этом и остановимся

In [9]:
import dill
with open("model.dill", "wb") as f:
    dill.dump(pipeline, f)